## Profile Uploader & Viewer API

### Required Installation

In [1]:
!pip install fastapi uvicorn python-multipart pillow jinja2 nest_asyncio aiofiles
!pip install anyio==3.6.2

#### nest_asyncio lets us run uvicorn server inside a Jupyter cell.

### Step 1: Imports and Setup

In [2]:
import os
import json
from uuid import uuid4
from fastapi import FastAPI, Form, File, UploadFile, Request, HTTPException
from fastapi.responses import HTMLResponse, FileResponse
from fastapi.templating import Jinja2Templates
from fastapi.staticfiles import StaticFiles
import nest_asyncio

### Step 2: Initialize App and Directories

In [3]:
app = FastAPI(title="📸 Profile Uploader API")

UPLOAD_DIR = "uploads"
DATA_FILE = "data/profiles.json"

os.makedirs(UPLOAD_DIR, exist_ok=True)
os.makedirs("data", exist_ok=True)
os.makedirs("templates", exist_ok=True)

# Mount static directory to serve uploaded images
app.mount("/uploads", StaticFiles(directory=UPLOAD_DIR), name="uploads")
templates = Jinja2Templates(directory="templates")

# Initialize the data file if not present
if not os.path.exists(DATA_FILE):
    with open(DATA_FILE, "w") as f:
        json.dump([], f)


### Step 3: Utility Functions

In [4]:
def save_profile(profile):
    with open(DATA_FILE, "r") as f:
        profiles = json.load(f)
    profiles.append(profile)
    with open(DATA_FILE, "w") as f:
        json.dump(profiles, f, indent=4)

def get_profiles():
    with open(DATA_FILE, "r") as f:
        return json.load(f)

### Step 4: API Routes

In [5]:
@app.get("/", response_class=HTMLResponse)
def home(request: Request):
    profiles = get_profiles()
    return templates.TemplateResponse("index.html", {"request": request, "profiles": profiles})

@app.post("/upload-profile/")
async def upload_profile(
    name: str = Form(...),
    bio: str = Form(...),
    image: UploadFile = File(...)
):
    ext = os.path.splitext(image.filename)[-1]
    image_id = f"{uuid4()}{ext}"
    image_path = os.path.join(UPLOAD_DIR, image_id)

    with open(image_path, "wb") as f:
        content = await image.read()
        f.write(content)

    profile = {
        "name": name,
        "bio": bio,
        "image_filename": image_id
    }
    save_profile(profile)
    return {"message": "Profile uploaded", "image_url": f"/uploads/{image_id}"}

@app.get("/image/{filename}")
def get_image(filename: str):
    path = os.path.join(UPLOAD_DIR, filename)
    if not os.path.exists(path):
        raise HTTPException(status_code=404, detail="Image not found")
    return FileResponse(path)

### Step 5: HTML Template (Simple Frontend Form)

In [6]:
html_code = """
<!DOCTYPE html>
<html>
<head><title>Upload Profile</title></head>
<body>
  <h2>Upload Profile</h2>
  <form action="/upload-profile/" enctype="multipart/form-data" method="post">
    Name: <input type="text" name="name"><br><br>
    Bio: <textarea name="bio"></textarea><br><br>
    Image: <input type="file" name="image"><br><br>
    <input type="submit" value="Upload">
  </form>

  <h2>Uploaded Profiles</h2>
  {% for profile in profiles %}
    <div style="margin-bottom: 20px;">
      <strong>{{ profile.name }}</strong><br>
      <em>{{ profile.bio }}</em><br>
      <img src="/uploads/{{ profile.image_filename }}" width="150"><br>
    </div>
  {% endfor %}
</body>
</html>
"""

with open("templates/index.html", "w") as f:
    f.write(html_code)

### Step 6: Launch API Server in Jupyter Notebook

In [ ]:
import uvicorn
nest_asyncio.apply()

# Launch server from notebook (run once per kernel)
uvicorn.run(app, host="0.0.0.0", port=8000)

INFO:     Started server process [8776]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


INFO:     127.0.0.1:52650 - "GET / HTTP/1.1" 200 OK
INFO:     127.0.0.1:52650 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     127.0.0.1:52658 - "POST /upload-profile/ HTTP/1.1" 200 OK
INFO:     127.0.0.1:52663 - "GET /upload-profile/ HTTP/1.1" 405 Method Not Allowed
INFO:     127.0.0.1:52663 - "GET / HTTP/1.1" 200 OK
INFO:     127.0.0.1:52663 - "GET /uploads/560aea48-fa69-4a77-b04c-7ae37ffdee71.png HTTP/1.1" 200 OK
INFO:     127.0.0.1:52664 - "POST /upload-profile/ HTTP/1.1" 200 OK
INFO:     127.0.0.1:52673 - "GET / HTTP/1.1" 200 OK
INFO:     127.0.0.1:52673 - "GET /uploads/f807c5f8-d00e-4c05-a42c-78d36d5d5a19.png HTTP/1.1" 200 OK
INFO:     127.0.0.1:52674 - "GET /uploads/560aea48-fa69-4a77-b04c-7ae37ffdee71.png HTTP/1.1" 304 Not Modified
INFO:     127.0.0.1:52679 - "GET / HTTP/1.1" 200 OK
INFO:     127.0.0.1:52679 - "GET /uploads/560aea48-fa69-4a77-b04c-7ae37ffdee71.png HTTP/1.1" 304 Not Modified
INFO:     127.0.0.1:52681 - "GET /uploads/f807c5f8-d00e-4c05-a42c-78d36d5d5a19.png 